ANALISIS DE DATOS

In [ ]:
import os

labels_dir = r"CHAD\CHAD_Meta\anomaly_labels"

count_normal = 0
count_anomalous = 0

for filename in os.listdir(labels_dir):
    if filename.endswith(".npy"):
        label = filename.split("_")[-1].split(".")[0]
        if label == "1":
            count_anomalous += 1
        elif label == "0":
            count_normal += 1

print(f"Archivos normales (0): {count_normal}")
print(f"Archivos anómalos (1): {count_anomalous}")
print(f"Total archivos: {count_normal + count_anomalous}")


Archivos normales (0): 278
Archivos anómalos (1): 134
Total archivos: 412


In [ ]:
import numpy as np
from pathlib import Path

example_file = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\CHAD\CHAD_Meta\anomaly_labels\1_071_1.npy")


print(f"Analizando archivo: {example_file.name}")

data = np.load(example_file, allow_pickle=True)

print("Tipo:", type(data))
print("Shape:", data.shape)

if isinstance(data, np.ndarray):
    print("Valores únicos:", np.unique(data))
    print("Primeros 20 valores:", data[:])

else:
    print("Contenido:")
    print(data)


Analizando archivo: 1_071_1.npy
Tipo: <class 'numpy.ndarray'>
Shape: (599,)
Valores únicos: [0. 1.]
Primeros 20 valores: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.

ANALIZAR KEYPOINTS POR VIDEO DE PRUEBA

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path

VIDEO_IN   = r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\DATASETS_COMBINADOS\anomalia\V_21.mp4"
VIDEO_OUT  = r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\video_keypoints.mp4"
MODEL_PATH = "yolo11m-pose.pt"   # correcto
IMGSZ      = 960
CONF_TH    = 0.25
IOU_TH     = 0.50
DRAW_CONF  = 0.20                             

SKELETON = [
    (0,1),(0,2),(1,3),(2,4),
    (5,6),(5,7),(7,9),(6,8),(8,10),
    (5,11),(6,12),(11,12),(11,13),(13,15),(12,14),(14,16)
]

COLORS = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,255),(0,255,255)]

# =============== DIBUJO ===============
def draw_poses(frame, kps_xy, kps_conf, skeleton=SKELETON, conf_th=DRAW_CONF):
    h, w = frame.shape[:2]
    for i in range(kps_xy.shape[0]):  # por persona
        color = COLORS[i % len(COLORS)]
        pts = kps_xy[i].astype(int)            # (17,2)
        conf = kps_conf[i] if kps_conf is not None else np.ones(17)

        # puntos
        for (x, y), c in zip(pts, conf):
            if c >= conf_th and 0 <= x < w and 0 <= y < h:
                cv2.circle(frame, (int(x), int(y)), 2, color, -1, cv2.LINE_AA)

        # líneas
        for a, b in skeleton:
            if conf[a] >= conf_th and conf[b] >= conf_th:
                xa, ya = pts[a]; xb, yb = pts[b]
                if 0 <= xa < w and 0 <= ya < h and 0 <= xb < w and 0 <= yb < h:
                    cv2.line(frame, (int(xa), int(ya)), (int(xb), int(yb)), color, 2, cv2.LINE_AA)

# =============== MAIN ===============
def main():
    cap = cv2.VideoCapture(VIDEO_IN)
    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir: {VIDEO_IN}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    writer = None
    if VIDEO_OUT:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        Path(VIDEO_OUT).parent.mkdir(parents=True, exist_ok=True)
        writer = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (W, H))

    model = YOLO(MODEL_PATH)

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        # Inference en el frame (numpy)
        res = model.predict(
            source=frame,
            imgsz=IMGSZ,
            conf=CONF_TH,
            iou=IOU_TH,
            verbose=False
        )[0]

        # Keypoints: (num_personas, 17, 2) y (num_personas, 17)
        kps_xy   = None
        kps_conf = None
        if res.keypoints is not None:
            # Ultralytics expone .xy y .confidence
            # Algunas versiones usan .conf; cubrimos ambos.
            kps_xy = res.keypoints.xy.cpu().numpy()                    # (N,17,2)
            kps_conf = getattr(res.keypoints, "confidence", None)
            if kps_conf is None:
                kps_conf = getattr(res.keypoints, "conf", None)
            kps_conf = kps_conf.cpu().numpy() if kps_conf is not None else None

        if kps_xy is not None and kps_xy.size > 0:
            draw_poses(frame, kps_xy, kps_conf, SKELETON, DRAW_CONF)

        cv2.imshow("YOLOv11 Pose Overlay", frame)
        if writer: writer.write(frame)

        key = cv2.waitKey(max(1, int(1000/fps))) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    if writer: writer.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [2]:
import cv2, numpy as np, torch, json
from ultralytics import YOLO
from pathlib import Path

# ========= CONFIG =========
VIDEO_PATH  = r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\1_069_1.mp4"
LABEL_NPY   = r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\anomaly_labels\1_069_1.npy"        # etiquetas 0/1 por frame (del anotador)
OUT_NPZ     = r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\key.npz"

MODEL       = "yolo11m-pose.pt"            # calidad>velocidad (cámbialo a 'yolo11s-pose.pt' si tu GPU es débil)
IMGSZ       = 960                           # 896–960 = keypoints más definidos
CONF_TH     = 0.25
IOU_TH      = 0.50
TOPK        = 5                             # máximo personas por frame que guardas

# Opcional: si sabes que tus labels empiezan en un frame distinto (offset en frames de VIDEO → labels)
START_OFFSET = 0                            # positivo: etiquetas comienzan después; negativo: antes

# ========= UTILS =========
def build_label_mapping(n_video_frames, labels, start_offset=0):
    """
    Devuelve:
      labels_aligned: len == n_video_frames
      map_idx: para cada frame de video (0..Fv-1), qué índice de label se usó (o -1 si fuera de rango)
    Regla: vecino más cercano con offset.
    """
    L = len(labels)
    map_idx = np.full(n_video_frames, -1, dtype=np.int32)

    if L <= 1:
        # Trivial: todo cero o único valor
        if L == 1:
            map_idx[:] = 0
        return (labels[map_idx.clip(0, L-1)] if L else np.zeros(n_video_frames, dtype=np.int32)), map_idx

    # mapea [0..Fv-1] → [0..L-1] con offset
    # índice proporcional + desplazamiento
    src = np.linspace(0, L - 1, num=n_video_frames)
    src = np.round(src).astype(int) + start_offset
    src = np.clip(src, 0, L - 1)
    map_idx = src
    labels_aligned = labels[map_idx]
    return labels_aligned.astype(labels.dtype), map_idx

# ========= MAIN =========
def main():
    # carga video y labels
    labels = np.load(LABEL_NPY)  # vector 0/1 por frame (o multiclase)
    cap = cv2.VideoCapture(VIDEO_PATH)
    assert cap.isOpened(), f"No se pudo abrir el video: {VIDEO_PATH}"

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    N   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # modelo
    model = YOLO(MODEL)
    if torch.cuda.is_available():
        model.to("cuda")
    model.fuse()

    # salida
    kps = np.full((N, TOPK, 17, 3), np.nan, dtype=np.float32)
    per_frame_counts = np.zeros(N, dtype=np.int32)

    # extrae
    idx = 0
    while True:
        ok, frame = cap.read()
        if not ok: break

        r = model.predict(
            frame, imgsz=IMGSZ, conf=CONF_TH, iou=IOU_TH,
            verbose=False, half=torch.cuda.is_available()
        )[0]

        if r.keypoints is not None and r.keypoints.xy is not None and r.keypoints.xy.shape[0] > 0:
            xy = r.keypoints.xy.cpu().numpy()  # (P,17,2)
            c  = getattr(r.keypoints, "confidence", None)
            if c is None: c = getattr(r.keypoints, "conf", None)
            c  = c.cpu().numpy() if c is not None else np.ones(xy.shape[:2], dtype=np.float32)

            # ordena personas por confianza media
            order = np.argsort(-c.mean(axis=1))
            xy, c = xy[order], c[order]
            P = min(xy.shape[0], TOPK)
            per_frame_counts[idx] = P

            # (x,y,conf)
            arr = np.concatenate([xy[:P], c[:P, :, None]], axis=-1)  # (P,17,3)
            kps[idx, :P] = arr.astype(np.float32)

        idx += 1

    cap.release()

    # alinear etiquetas al número de frames del VIDEO
    labels_aligned, map_idx = build_label_mapping(N, labels, start_offset=START_OFFSET)

    meta = {
        "video_path": str(VIDEO_PATH),
        "labels_path": str(LABEL_NPY),
        "fps_video": float(fps),
        "width": int(W),
        "height": int(H),
        "n_video_frames": int(N),
        "labels_len": int(len(labels)),
        "start_offset": int(START_OFFSET),
        "imgsz": int(IMGSZ),
        "model": MODEL,
        "conf_th": float(CONF_TH),
        "iou_th": float(IOU_TH),
        "topk": int(TOPK)
    }

    Path(OUT_NPZ).parent.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(
        OUT_NPZ,
        kps=kps,
        per_frame_counts=per_frame_counts,
        labels_aligned=labels_aligned,
        map_label_idx=map_idx,
        meta=json.dumps(meta)
    )
    print(f"✓ Guardado {OUT_NPZ}")
    print(f"kps: {kps.shape} | frames: {N} | labels_in: {len(labels)} | offset: {START_OFFSET}")

if __name__ == "__main__":
    main()


YOLO11m-pose summary (fused): 134 layers, 20,889,004 parameters, 0 gradients, 71.4 GFLOPs
✓ Guardado C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\key.npz
kps: (1199, 5, 17, 3) | frames: 1199 | labels_in: 1199 | offset: 0


PROBAR POR NPZ

In [17]:
import numpy as np, json

NPZ_PATH = r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz_fall\video (5).npz"
z = np.load(NPZ_PATH, allow_pickle=True)

kps = z["kps"]                      # [F, K, 17, 3] (x,y,conf) con NaN
counts = z["per_frame_counts"]      # [F]
labels = z.get("labels_aligned", None)  # opcional
map_idx = z.get("map_label_idx", None)  # opcional
meta = json.loads(z["meta"].item() if hasattr(z["meta"], "item") else z["meta"].tolist())

print("kps:", kps.shape, "dtype:", kps.dtype)
print("per_frame_counts:", counts.shape, "sum personas:", int(counts.sum()))
if labels is not None: print("labels_aligned:", labels.shape, "positivos:", int(labels.sum()))
print("meta:", meta)

# Chequeos rápidos
F, K = kps.shape[:2]
assert counts.shape[0] == F, "counts desalineado con frames"
assert np.isfinite(kps[..., :2][~np.isnan(kps[..., :2])]).all(), "x/y con inf"
print("✓ NPZ ok a nivel de forma/básicos")


kps: (181, 1, 17, 3) dtype: float32
per_frame_counts: (181,) sum personas: 181
labels_aligned: (181,) positivos: 33
meta: {'video_path': 'C:\\Users\\Usuario\\Documents\\GitHub\\tesis-deteccionar-sistema\\Dataset_Le21+GMDCSA24\\VideosFall\\video (5).avi', 'labels_path': 'C:\\Users\\Usuario\\Documents\\GitHub\\tesis-deteccionar-sistema\\Dataset_Le21+GMDCSA24\\Annotation_files\\video (5).txt', 'fps_video': 25.0, 'width': 320, 'height': 240, 'n_video_frames': 181, 'labels_len': 181, 'imgsz': 960, 'model': 'yolo11m-pose.pt', 'conf_th': 0.25, 'iou_th': 0.6, 'topk': 1}
✓ NPZ ok a nivel de forma/básicos


In [ ]:
import cv2, numpy as np, random

SKELETON = [(0,1),(0,2),(1,3),(2,4),(5,6),(5,7),(7,9),(6,8),(8,10),
            (5,11),(6,12),(11,12),(11,13),(13,15),(12,14),(14,16)]

def draw_frame_from_npz(canvas, kps_frame, draw_conf=0.2):
    H, W = canvas.shape[:2]
    colors = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,255)]
    for p in range(kps_frame.shape[0]):
        color = colors[p % len(colors)]
        pts = kps_frame[p, :, :2]
        conf = kps_frame[p, :, 2]
        if np.isnan(pts).all(): continue
        # puntos
        for (x,y), c in zip(pts, conf):
            if not np.isnan(x) and not np.isnan(y) and c >= draw_conf and 0<=x<W and 0<=y<H:
                cv2.circle(canvas, (int(x),int(y)), 2, color, -1, cv2.LINE_AA)
        # líneas
        for a,b in SKELETON:
            xa,ya = pts[a]; xb,yb = pts[b]
            ca,cb = conf[a], conf[b]
            if (not np.isnan([xa,ya,xb,yb]).any() and ca>=draw_conf and cb>=draw_conf and
                0<=xa<W and 0<=ya<H and 0<=xb<W and 0<=yb<H):
                cv2.line(canvas, (int(xa),int(ya)), (int(xb),int(yb)), color, 2, cv2.LINE_AA)
    return canvas

W, H = int(meta["width"]), int(meta["height"])
f = random.randrange(kps.shape[0])
canvas = np.zeros((H, W, 3), np.uint8)
img = draw_frame_from_npz(canvas, kps[f])
cv2.imshow(f"Frame {f}", img); cv2.waitKey(0); cv2.destroyAllWindows()


In [8]:
import cv2, numpy as np

VIDEO_PATH = meta.get("video_path", "")  # lo guardaste en meta
cap = cv2.VideoCapture(VIDEO_PATH) if VIDEO_PATH else None
F = kps.shape[0]
draw_conf = 0.2

def draw_on(frame, kps_frame):
    return draw_frame_from_npz(frame, kps_frame, draw_conf)

i = 0
while True:
    if cap is not None:
        ok, frame = cap.read()
        if not ok or i>=F: break
    else:
        # Sin video: lienzo negro con tamaño meta
        frame = np.zeros((int(meta["height"]), int(meta["width"]), 3), np.uint8)
        if i>=F: break

    frame = draw_on(frame, kps[i])

    # Mostrar etiqueta si existe
    if labels is not None:
        lab = int(labels[i])
        txt = "ANOMALIA" if lab==1 else "NORMAL"
        color = (0,0,255) if lab==1 else (0,200,0)
        cv2.putText(frame, f"{txt}", (20,60), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3, cv2.LINE_AA)
    cv2.putText(frame, f"Frame {i}", (20,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    cv2.imshow("NPZ overlay", frame)
    if cv2.waitKey(1) & 0xFF==ord('q'): break
    i += 1

if cap is not None: cap.release()
cv2.destroyAllWindows()


SE TERMINA LAS VISUALIZACIONES POR NPZ

In [2]:
# inspect_one_npz.py
import numpy as np, json

NPZ_PATH = r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz\1_002_0.npz"  # <-- cambia

def main():
    z = np.load(NPZ_PATH, allow_pickle=True)
    print(f"\n=== {NPZ_PATH} ===")
    print("Claves:", list(z.files))

    # kps: [F, K, 17, 3] -> (x, y, conf_joint)
    if "kps" in z:
        kps = z["kps"]
        print(f"\n[kps] shape={kps.shape} dtype={kps.dtype}")
        F, K, J, C = kps.shape
        conf = kps[..., 2]
        conf_finite = conf[np.isfinite(conf)]
        if conf_finite.size:
            print(f"  conf min/max: {conf_finite.min():.4f} / {conf_finite.max():.4f}")
        # ejemplo rápido del frame 0, persona 0 (si existe)
        f0 = 0
        if F and K:
            p0_valid = ~np.isnan(kps[f0, 0, :, 0]).all()
            if p0_valid:
                nose = kps[f0, 0, 0]     # joint 0 (nariz)
                print(f"  Ejemplo f{f0}, p0, nose (x,y,conf) = {nose.tolist()}")

    # labels_aligned: [F]
    if "labels_aligned" in z:
        labels = z["labels_aligned"]
        uniq, cnt = np.unique(labels, return_counts=True)
        ratio = float(labels.mean()) if labels.size else 0.0
        print(f"\n[labels_aligned] shape={labels.shape} dtype={labels.dtype}")
        print(f"  únicos={uniq.tolist()}  counts={cnt.tolist()}  pos_ratio={ratio:.4f}")

    # per_frame_counts: [F]
    if "per_frame_counts" in z:
        c = z["per_frame_counts"]
        if c.size:
            print(f"\n[per_frame_counts] shape={c.shape}  mean={c.mean():.3f}  p50={np.percentile(c,50):.0f}  p95={np.percentile(c,95):.0f}")

    # meta (json)
    if "meta" in z:
        meta_raw = z["meta"]
        try:
            meta = json.loads(meta_raw.item() if hasattr(meta_raw, "item") else meta_raw)
        except Exception:
            meta = {}
        keep = ["video_path","labels_path","fps_video","width","height","n_video_frames","labels_len","imgsz","model","topk"]
        print("\n[meta]")
        for k in keep:
            if k in meta:
                print(f"  {k}: {meta[k]}")

if __name__ == "__main__":
    main()



=== C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz\1_002_0.npz ===
Claves: ['kps', 'per_frame_counts', 'labels_aligned', 'meta']

[kps] shape=(5399, 5, 17, 3) dtype=float32
  conf min/max: 0.0011 / 1.0000
  Ejemplo f0, p0, nose (x,y,conf) = [899.5, 308.5, 0.57470703125]

[labels_aligned] shape=(5399,) dtype=float64
  únicos=[0.0]  counts=[5399]  pos_ratio=0.0000

[per_frame_counts] shape=(5399,)  mean=1.828  p50=2  p95=3

[meta]
  video_path: C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\CHAD_VIDEOS\1_002_0.mp4
  labels_path: C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\anomaly_labels\1_002_0.npy
  fps_video: 29.97002997002997
  width: 1920
  height: 1080
  n_video_frames: 5399
  labels_len: 5399
  imgsz: 960
  model: yolo11m-pose.pt
  topk: 5


In [3]:
import numpy as np
from pathlib import Path

# ==== CONFIG ====
NPZ_DIR  = r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz"
SEQ_LEN  = 32
STRIDE   = 8
TAU      = 0.50
MIN_RUN  = 3

def has_run(x, k=3):
    c = 0
    for v in x:
        c = c + 1 if v == 1 else 0
        if c >= k: return True
    return False

def window_labels_from_frames(labels, T, stride, tau, min_run):
    """Devuelve y_win (0/1) por ventana sin usar features (solo etiquetas frame a frame)."""
    F = len(labels)
    ys = []
    for s in range(0, F - T + 1, stride):
        seg = labels[s:s+T]
        is_pos = (seg.mean() >= tau) and has_run(seg, min_run)
        ys.append(1 if is_pos else 0)
    return np.array(ys, dtype=np.int64)

def main():
    paths = sorted(Path(NPZ_DIR).glob("*.npz"))
    assert paths, f"Sin .npz en {NPZ_DIR}"

    tot_frames = 0
    tot_pos_frames = 0
    tot_win = 0
    tot_pos_win = 0

    for p in paths:
        z = np.load(p, allow_pickle=True)
        if "labels_aligned" not in z:
            print(f"✗ {p.name} sin 'labels_aligned' → salto")
            continue
        y = z["labels_aligned"].astype(np.int64)
        pos_ratio = float(y.mean()) if y.size else 0.0

        y_win = window_labels_from_frames(y, SEQ_LEN, STRIDE, TAU, MIN_RUN)
        n_pos_w = int(y_win.sum())
        n_win   = int(len(y_win))
        n_neg_w = n_win - n_pos_w

        print(f"{p.name:30s} | frames={len(y):5d}  pos_ratio={pos_ratio:0.4f}  "
              f"ventanas: pos={n_pos_w:5d} neg={n_neg_w:5d} tot={n_win:5d}")

        tot_frames += len(y)
        tot_pos_frames += int(y.sum())
        tot_win += n_win
        tot_pos_win += n_pos_w

    print("\n=== RESUMEN GLOBAL ===")
    if tot_frames:
        print(f"Frames: tot={tot_frames}  pos={tot_pos_frames}  pos_ratio={tot_pos_frames/tot_frames:0.4f}")
    if tot_win:
        print(f"Ventanas: tot={tot_win}  pos={tot_pos_win}  neg={tot_win - tot_pos_win}  "
              f"pos_ratio={tot_pos_win/tot_win:0.4f}")

if __name__ == "__main__":
    main()


1_002_0.npz                    | frames= 5399  pos_ratio=0.0000  ventanas: pos=    0 neg=  671 tot=  671
1_003_0.npz                    | frames= 6299  pos_ratio=0.0000  ventanas: pos=    0 neg=  784 tot=  784
1_033_0.npz                    | frames= 1799  pos_ratio=0.0000  ventanas: pos=    0 neg=  221 tot=  221
1_069_1.npz                    | frames= 1199  pos_ratio=0.7314  ventanas: pos=  110 neg=   36 tot=  146
1_070_1.npz                    | frames=  449  pos_ratio=0.4410  ventanas: pos=   25 neg=   28 tot=   53
1_071_1.npz                    | frames=  599  pos_ratio=0.5376  ventanas: pos=   40 neg=   31 tot=   71
1_072_1.npz                    | frames=  749  pos_ratio=0.8198  ventanas: pos=   77 neg=   13 tot=   90
1_073_1.npz                    | frames=  449  pos_ratio=0.3341  ventanas: pos=   19 neg=   34 tot=   53
1_074_1.npz                    | frames=  449  pos_ratio=0.2294  ventanas: pos=   13 neg=   40 tot=   53
1_075_1.npz                    | frames=  449  pos_rati

EXTRAER NPZ POR VIDEO Y KEYPOINTS

////////////////dataset chad

In [ ]:
# batch_npz_cuda_tqdm.py  (versión con conf por joint)
import json, cv2, numpy as np, torch
from pathlib import Path
from ultralytics import YOLO
from tqdm.auto import tqdm

# =============== CONFIG ===============
VIDEOS_DIR = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\CHAD_VIDEOS")
LABELS_DIR = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\anomaly_labels")
OUT_DIR    = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz")

MODEL      = "yolo11m-pose.pt"
IMGSZ      = 960
CONF_TH    = 0.25
IOU_TH     = 0.50
TOPK       = 3                    # máx. personas por frame
VIDEO_EXTS = {".mp4", ".avi", ".mov", ".mkv", ".m4v"}

# =============== HELPERS ===============
def _get_xy_conf_per_joint(result):
    """
    Devuelve (xy, c) con shapes:
      xy: [P, 17, 2], c: [P, 17]  (confianza por joint)
    Usa la confianza del modelo si existe; si no, deriva una máscara 0/1 por joint (validez geométrica).
    """
    kp = result.keypoints
    if kp is None or kp.xy is None or kp.xy.shape[0] == 0:
        return None, None

    xy = kp.xy.detach().cpu().numpy()  # [P,17,2]

    # Intenta leer confianza por joint de Ultralytics (distintas versiones exponen .conf o .confidence)
    c_attr = None
    if hasattr(kp, "conf") and (kp.conf is not None):
        c_attr = kp.conf
    elif hasattr(kp, "confidence") and (kp.confidence is not None):
        c_attr = kp.confidence

    if c_attr is not None:
        c = c_attr.detach().cpu().numpy()  # [P,17]
        # clamp por seguridad
        c = np.clip(c, 0.0, 1.0).astype(np.float32)
    else:
        # No hay conf por joint: derivar una máscara por joint (0/1) basada en validez de (x,y)
        # Nota: >1 px para evitar falsos (0,0), y finitos.
        valid = (
            np.isfinite(xy[..., 0]) & np.isfinite(xy[..., 1]) &
            (xy[..., 0] > 1) & (xy[..., 1] > 1)
        )
        c = valid.astype(np.float32)  # [P,17]

    return xy.astype(np.float32), c.astype(np.float32)

# =============== FUNCIONES ===============
def process_one(video_path: Path, labels_path: Path, model: YOLO, out_npz: Path, use_cuda: bool):
    # Verifica labels
    if not labels_path.exists():
        tqdm.write(f"✗ SIN labels → {labels_path.name} no existe. Salto {video_path.name}")
        return False

    labels = np.load(str(labels_path))

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        tqdm.write(f"✗ No se pudo abrir video: {video_path}")
        return False

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    N   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Validación 1:1
    if N != len(labels):
        tqdm.write(f"✗ Descuadre frames/labels en {video_path.name}: frames={N} vs labels={len(labels)}. Salto.")
        cap.release()
        return False

    # Buffers
    kps = np.full((N, TOPK, 17, 3), np.nan, dtype=np.float32)  # (x,y,conf_joint)
    per_frame_counts = np.zeros(N, dtype=np.int32)

    # Progreso por frames
    pbar = tqdm(total=N, desc=f"{video_path.stem}", leave=False)

    idx = 0
    while idx < N:
        ok, frame = cap.read()
        if not ok:
            break

        r = model.predict(
            frame,
            imgsz=IMGSZ,
            conf=CONF_TH,
            iou=IOU_TH,
            verbose=False,
            half=use_cuda  # FP16 en GPU
        )[0]

        xy, c = _get_xy_conf_per_joint(r)
        if xy is not None and c is not None and xy.shape[0] > 0:
            # Ordenar personas por confianza media (por joint)
            order = np.argsort(-c.mean(axis=1))
            xy, c = xy[order], c[order]

            P = min(xy.shape[0], TOPK)
            per_frame_counts[idx] = P

            # Ensamblar (x,y,conf_joint) por joint
            arr = np.concatenate([xy[:P], c[:P, :, None]], axis=-1)  # [P,17,3]
            # Clamp de coords a bordes de imagen (por robustez de validez futura)
            arr[..., 0] = np.clip(arr[..., 0], 0, W - 1)
            arr[..., 1] = np.clip(arr[..., 1], 0, H - 1)

            kps[idx, :P] = arr.astype(np.float32)

        idx += 1
        pbar.update(1)

    pbar.close()
    cap.release()

    # Guardado
    meta = {
        "video_path": str(video_path),
        "labels_path": str(labels_path),
        "fps_video": float(fps),
        "width": int(W),
        "height": int(H),
        "n_video_frames": int(N),
        "labels_len": int(len(labels)),
        "imgsz": int(IMGSZ),
        "model": MODEL,
        "conf_th": float(CONF_TH),
        "iou_th": float(IOU_TH),
        "topk": int(TOPK)
    }

    out_npz.parent.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(
        out_npz,
        kps=kps,
        per_frame_counts=per_frame_counts,
        labels_aligned=labels.astype(labels.dtype),
        meta=json.dumps(meta)
    )
    tqdm.write(f"✓ {video_path.name} → {out_npz.name} | frames={N} | personas={int(per_frame_counts.sum())}")
    return True

def main():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    # Modelo una sola vez
    model = YOLO(MODEL)
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        model.to("cuda")
        torch.backends.cudnn.benchmark = True  # tamaño fijo → mejor rendimiento
    model.fuse()

    videos = sorted([p for p in VIDEOS_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS])
    if not videos:
        print(f"✗ No hay videos en {VIDEOS_DIR}")
        return

    ok_cnt, fail_cnt = 0, 0
    for v in tqdm(videos, desc="Videos", unit="vid"):
        stem = v.stem
        labels_path = LABELS_DIR / f"{stem}.npy"
        out_npz = OUT_DIR / f"{stem}.npz"

        if out_npz.exists():
            tqdm.write(f"- {stem}: ya existe, salto.")
            continue

        try:
            if process_one(v, labels_path, model, out_npz, use_cuda):
                ok_cnt += 1
            else:
                fail_cnt += 1
        except Exception as e:
            tqdm.write(f"  ✗ error en {stem}: {e}")
            fail_cnt += 1

    print(f"\nListo. OK={ok_cnt} | Fallos={fail_cnt} | Out={OUT_DIR} | CUDA={'sí' if use_cuda else 'no'}")

if __name__ == "__main__":
    main()


////////////////dataset caidas

In [ ]:
# make_npz_fall_yolo11.py
import cv2, numpy as np, torch, json
from pathlib import Path
from ultralytics import YOLO
from tqdm.auto import tqdm

# ========= CONFIG =========
VIDEOS_DIR = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\Dataset_Le21+GMDCSA24\VideosFall")
ANN_DIR    = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\Dataset_Le21+GMDCSA24\Annotation_files")
OUT_DIR    = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz_fall")

MODEL      = "yolo11m-pose.pt"   # o "yolo11s-pose.pt" si priorizas FPS
IMGSZ      = 960
CONF_TH    = 0.25
IOU_TH     = 0.50
TOPK       = 3                    # máx. personas por frame
VIDEO_EXTS = {".mp4", ".avi", ".mov", ".mkv", ".m4v"}

# ========= UTILS =========
def read_range_txt(txt_path: Path, n_frames: int):
    """
    Lee 2 líneas: start, end (frames). Soporta 0-based o 1-based.
    Devuelve labels [F] con 1 en [start, end], inclusive y recortado.
    """
    with open(txt_path, "r", encoding="utf-8") as f:
        lines = [l.strip() for l in f if l.strip() != ""]
    if len(lines) < 2:
        # sin rango válido => todo 0
        return np.zeros(n_frames, dtype=np.uint8)

    start_raw = int(float(lines[0]))
    end_raw   = int(float(lines[1]))

    # Detecta 1-based y corrige si hace falta
    start, end = start_raw, end_raw
    if end >= n_frames and (end - 1) < n_frames:
        start, end = start_raw - 1, end_raw - 1

    # Clamp a [0, n_frames-1] y ordena
    start = max(0, min(n_frames - 1, start))
    end   = max(0, min(n_frames - 1, end))
    if end < start:
        start, end = end, start

    y = np.zeros(n_frames, dtype=np.uint8)
    y[start:end + 1] = 1
    return y

def process_one(video_path: Path, ann_path: Path, model: YOLO, out_npz: Path, use_cuda: bool):
    if not ann_path.exists():
        tqdm.write(f"✗ Sin anotación: {ann_path.name} — salto {video_path.name}")
        return False

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        tqdm.write(f"✗ No se pudo abrir video: {video_path}")
        return False

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    N   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    labels = read_range_txt(ann_path, N)

    # Buffers
    kps = np.full((N, TOPK, 17, 3), np.nan, dtype=np.float32)
    per_frame_counts = np.zeros(N, dtype=np.int32)

    pbar = tqdm(total=N, desc=video_path.stem, leave=False)
    idx = 0
    while idx < N:
        ok, frame = cap.read()
        if not ok:
            break

        r = model.predict(frame, imgsz=IMGSZ, conf=CONF_TH, iou=IOU_TH,
                          verbose=False, half=use_cuda)[0]

        if r.keypoints is not None and r.keypoints.xy is not None and r.keypoints.xy.shape[0] > 0:
            xy = r.keypoints.xy.detach().cpu().numpy()  # (P,17,2)
            c  = None
            if hasattr(r.keypoints, "conf") and r.keypoints.conf is not None:
                c = r.keypoints.conf
            elif hasattr(r.keypoints, "confidence") and r.keypoints.confidence is not None:
                c = r.keypoints.confidence
            c = c.detach().cpu().numpy() if c is not None else np.ones(xy.shape[:2], dtype=np.float32)

            # orden por confianza media de joints
            order = np.argsort(-c.mean(axis=1))
            xy, c = xy[order], c[order]

            P = min(xy.shape[0], TOPK)
            per_frame_counts[idx] = P
            kps[idx, :P] = np.concatenate([xy[:P], c[:P, :, None]], axis=-1).astype(np.float32)

        idx += 1
        pbar.update(1)

    pbar.close()
    cap.release()

    # meta y guardado (mismo formato que CHAD)
    meta = {
        "video_path": str(video_path),
        "labels_path": str(ann_path),
        "fps_video": float(fps),
        "width": int(W),
        "height": int(H),
        "n_video_frames": int(N),
        "labels_len": int(len(labels)),
        "imgsz": int(IMGSZ),
        "model": MODEL,
        "conf_th": float(CONF_TH),
        "iou_th": float(IOU_TH),
        "topk": int(TOPK)
    }

    out_npz.parent.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(out_npz,
                        kps=kps,
                        per_frame_counts=per_frame_counts,
                        labels_aligned=labels.astype(labels.dtype),
                        meta=json.dumps(meta))
    tqdm.write(f"✓ {video_path.name} → {out_npz.name} | frames={N} | personas={int(per_frame_counts.sum())}")
    return True

# ========= MAIN =========
def main():
    torch.backends.cudnn.benchmark = True
    if torch.cuda.is_available():
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

    OUT_DIR.mkdir(parents=True, exist_ok=True)

    model = YOLO(MODEL)
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        model.to("cuda")
    model.fuse()

    videos = sorted([p for p in VIDEOS_DIR.iterdir() if p.suffix.lower() in VIDEO_EXTS])
    if not videos:
        print(f"✗ No hay videos en {VIDEOS_DIR}")
        return

    ok_cnt, fail_cnt = 0, 0
    for v in tqdm(videos, desc="Videos", unit="vid"):
        stem = v.stem
        ann_path = ANN_DIR / f"{stem}.txt"
        out_npz = OUT_DIR / f"{stem}.npz"

        if out_npz.exists():
            tqdm.write(f"- {stem}: ya existe, salto.")
            continue

        try:
            if process_one(v, ann_path, model, out_npz, use_cuda):
                ok_cnt += 1
            else:
                fail_cnt += 1
        except Exception as e:
            tqdm.write(f"  ✗ error en {stem}: {e}")
            fail_cnt += 1

    print(f"\nListo. OK={ok_cnt} | Fallos={fail_cnt} | Out={OUT_DIR} | CUDA={'sí' if use_cuda else 'no'}")

if __name__ == "__main__":
    main()


////////////////dataset peleas

In [103]:
# batch_npz_videolevel_mix.py
import json, cv2, numpy as np, torch
from pathlib import Path
from ultralytics import YOLO
from tqdm.auto import tqdm

# ====== CONFIG ======
ANOMALY_DIR = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\DATASETS_COMBINADOS\anomalia")   # etiqueta dura = 1
NORMAL_DIR  = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\DATASETS_COMBINADOS\no_anomalia")     # etiqueta dura = 0

# Puedes añadir más carpetas (dir, label) aquí:
SOURCES = [
    (ANOMALY_DIR, 1),
    (NORMAL_DIR,  0),
]

OUT_DIR   = Path(r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz_video")
MODEL     = "yolo11m-pose.pt"     
IMGSZ      = 960
CONF_TH    = 0.25
IOU_TH     = 0.50
TOPK       = 3                    # máx. personas por frame
VIDEO_EXT = {".mp4", ".avi", ".mov", ".mkv", ".m4v"}
RECURSIVE = True                  # si True, usa rglob para recorrer subcarpetas
TRIM_BORDERS_POS = 5             # recorta 15 frames al inicio/fin SOLO si label=1 (clips duros)
MIN_FRAMES = 8                    # descarta videos muy cortos

# ====== HELPERS ======
def _get_xy_conf_per_joint(result):
    """
    Devuelve:
      xy: [P,17,2]  (float32)
      c : [P,17]    (float32, confianza por joint en [0,1])
    Si la versión de Ultralytics no trae conf por joint, se crea máscara 0/1 por validez geométrica.
    """
    kp = result.keypoints
    if kp is None or kp.xy is None or kp.xy.shape[0] == 0:
        return None, None

    xy = kp.xy.detach().cpu().numpy().astype(np.float32)  # [P,17,2]

    c_attr = None
    if hasattr(kp, "conf") and kp.conf is not None:
        c_attr = kp.conf
    elif hasattr(kp, "confidence") and kp.confidence is not None:
        c_attr = kp.confidence

    if c_attr is not None:
        c = c_attr.detach().cpu().numpy().astype(np.float32)  # [P,17]
        c = np.clip(c, 0.0, 1.0)
    else:
        valid = (
            np.isfinite(xy[..., 0]) & np.isfinite(xy[..., 1]) &
            (xy[..., 0] > 1) & (xy[..., 1] > 1)
        )
        c = valid.astype(np.float32)  # [P,17]

    return xy, c

def _iter_videos(d: Path):
    if not d.exists():
        tqdm.write(f"Carpeta no existe: {d}")
        return []
    if RECURSIVE:
        return [p for p in d.rglob("*") if p.suffix.lower() in VIDEO_EXT]
    else:
        return [p for p in d.iterdir() if p.suffix.lower() in VIDEO_EXT]

# ====== CORE ======
def process_one(video_path: Path, model: YOLO, out_npz: Path, label_value: int, use_cuda: bool) -> bool:
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        tqdm.write(f"✗ No abre: {video_path.name}")
        return False

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    W   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    N   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if N < MIN_FRAMES:
        tqdm.write(f"- {video_path.name}: muy corto ({N} frames). Salto.")
        cap.release()
        return False

    # buffers
    kps = np.full((N, TOPK, 17, 3), np.nan, dtype=np.float32)  # (x,y,conf_joint)
    per_frame_counts = np.zeros(N, dtype=np.int32)

    # etiqueta dura por frame (con recorte en positivos para evitar bordes ruidosos)
    labels_aligned = np.full(N, int(label_value), dtype=np.int8)
    if label_value == 1 and TRIM_BORDERS_POS > 0 and N > 2 * TRIM_BORDERS_POS:
        labels_aligned[:TRIM_BORDERS_POS] = 0
        labels_aligned[-TRIM_BORDERS_POS:] = 0

    pbar = tqdm(total=N, desc=video_path.stem, leave=False)
    idx = 0
    while idx < N:
        ok, frame = cap.read()
        if not ok:
            break

        r = model.predict(frame, imgsz=IMGSZ, conf=CONF_TH, iou=IOU_TH,
                          verbose=False, half=use_cuda)[0]

        xy, c = _get_xy_conf_per_joint(r)
        if xy is not None and c is not None and xy.shape[0] > 0:
            # Ordenar personas por confianza media en joints
            order = np.argsort(-c.mean(axis=1))
            xy, c = xy[order], c[order]

            P = min(xy.shape[0], TOPK)
            per_frame_counts[idx] = P

            arr = np.concatenate([xy[:P], c[:P, :, None]], axis=-1).astype(np.float32)  # [P,17,3]
            # clamp coords
            arr[..., 0] = np.clip(arr[..., 0], 0, W - 1)
            arr[..., 1] = np.clip(arr[..., 1], 0, H - 1)

            kps[idx, :P] = arr

        idx += 1
        pbar.update(1)

    pbar.close()
    cap.release()

    meta = {
        "video_path": str(video_path),
        "labels_path": "video_level_label",
        "fps_video": float(fps),
        "width": int(W),
        "height": int(H),
        "n_video_frames": int(N),
        "labels_len": int(N),
        "imgsz": int(IMGSZ),
        "model": MODEL,
        "conf_th": float(CONF_TH),
        "iou_th": float(IOU_TH),
        "topk": int(TOPK),
        "video_level_label": int(label_value)
    }

    out_npz.parent.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(
        out_npz,
        kps=kps,
        per_frame_counts=per_frame_counts,
        labels_aligned=labels_aligned,
        meta=json.dumps(meta)
    )
    tqdm.write(f"✓ {video_path.name} → {out_npz.name} | frames={N} | personas={int(per_frame_counts.sum())}")
    return True

def main():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    # modelo una sola vez
    model = YOLO(MODEL)
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        model.to("cuda")
        torch.backends.cudnn.benchmark = True
    model.fuse()

    vids = []
    for d, y in SOURCES:
        vids_dir = _iter_videos(d)
        vids.extend([(p, y) for p in sorted(vids_dir)])

    if not vids:
        print("✗ No se encontraron videos.")
        return

    ok, fail = 0, 0
    for vpath, ylab in tqdm(vids, desc="Videos", unit="vid"):
        out_npz = OUT_DIR / f"{vpath.stem}.npz"
        if out_npz.exists():
            tqdm.write(f"- {vpath.stem}: ya existe, salto.")
            continue
        try:
            if process_one(vpath, model, out_npz, ylab, use_cuda):
                ok += 1
            else:
                fail += 1
        except Exception as e:
            tqdm.write(f"  ✗ error en {vpath.stem}: {e}")
            fail += 1

    print(f"\nListo. OK={ok} | Fallos={fail} | Out={OUT_DIR} | CUDA={'sí' if use_cuda else 'no'}")

if __name__ == "__main__":
    main()


c:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


YOLO11m-pose summary (fused): 134 layers, 20,889,004 parameters, 0 gradients, 71.4 GFLOPs


Videos:   2%|▏         | 1/64 [00:04<04:34,  4.36s/vid]

✓ 2.mp4 → 2.npz | frames=132 | personas=393


Videos:   3%|▎         | 2/64 [00:06<03:16,  3.17s/vid]

...

✓ t_n014_converted.avi → t_n014_converted.npz | frames=124 | personas=368


Videos:  98%|█████████▊| 63/64 [04:05<00:03,  3.08s/vid]

✓ t_n015_converted.avi → t_n015_converted.npz | frames=134 | personas=401


Videos: 100%|██████████| 64/64 [04:07<00:00,  3.87s/vid]

✓ t_n016_converted.avi → t_n016_converted.npz | frames=76 | personas=228

Listo. OK=64 | Fallos=0 | Out=C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz_video | CUDA=sí


ENTRENAMIENTO

In [ ]:
# mix_train_cnnlstm_focalloss_v2.py
import os, json, numpy as np, tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
from tensorflow.keras.losses import BinaryFocalCrossentropy
from pathlib import Path
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_auc_score, average_precision_score

# =======================
# CONFIG
# =======================
NPZ_DIRS = [
    r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz",          # CHAD (frame-level)
    r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz_fall",     # FALL (rango/frame-level)
    r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz_video"     # video-level (clips duros)
]

MODEL_KIND   = "cnn_lstm"   # "lstm" o "cnn_lstm"
SEQ_LEN      = 32
STRIDE       =12

# Limpieza/etiquetado
MIN_VIS_FRAC = 0.30   
CONF_MIN     = 0.10   
TRIM_BORDERS = 0     

VAL_RATIO    = 0.15   # split por VIDEO y por DOMINIO (sin solape)
TEST_RATIO   = 0.15
EPOCHS       = 30
BATCH_SIZE   = 124
SEED         = 42

MODEL_DIR = Path("./models_mix")
MODEL_DIR.mkdir(parents=True, exist_ok=True)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# =======================
# HELPERS
# =======================
def load_meta(z):
    meta_raw = z["meta"]
    try:
        return json.loads(meta_raw.item() if hasattr(meta_raw, "item") else meta_raw)
    except Exception:
        return {}

def pool_frame_to_51(kps_f, W, H):
    """
    kps_f: (K,17,3) -> (51,) normalizado. Toma por joint la persona con mayor conf.
    """
    out = np.zeros((17, 3), dtype=np.float32)
    conf_j = kps_f[..., 2]
    conf_j = np.nan_to_num(conf_j, nan=0.0)
    for j in range(17):
        idx = np.argmax(conf_j[:, j]) if conf_j.shape[0] else None
        if idx is not None and conf_j[idx, j] > 0:
            x, y, c = kps_f[idx, j, :]
            if np.isfinite(x) and np.isfinite(y):
                out[j, 0] = np.clip(x / max(W, 1), 0.0, 1.0)
                out[j, 1] = np.clip(y / max(H, 1), 0.0, 1.0)
                out[j, 2] = float(np.clip(c, 0.0, 1.0))
    return out.reshape(-1)

def frame_visible(kps_f, conf_min=CONF_MIN):
    conf = kps_f[..., 2]
    conf = np.nan_to_num(conf, nan=0.0)
    return bool((conf >= conf_min).any())

def video_to_windows(npz_path: Path):
    """
    Devuelve:
      Xw: (Nw, SEQ_LEN, 51)
      Yw: (Nw,)  -> etiqueta del ÚLTIMO frame (tiempo real)
      vid_id: nombre del video (string)
      domain: dominio (carpeta padre inmediata del .npz)
    Filtros:
      - Recorte de bordes para clips duros
      - Rechazo de ventanas con visibilidad < MIN_VIS_FRAC
    """
    z = np.load(npz_path, allow_pickle=True)
    if not {"kps", "labels_aligned", "meta"}.issubset(z.files):
        return np.zeros((0, SEQ_LEN, 51), np.float32), np.zeros((0,), np.int64), npz_path.name, npz_path.parent.name

    kps  = z["kps"]                               # (F,K,17,3)
    y    = z["labels_aligned"].astype(np.int64)   # (F,)
    meta = load_meta(z)
    W, H = int(meta.get("width", 1920)), int(meta.get("height", 1080))
    F    = kps.shape[0]

    if F != len(y) or F < SEQ_LEN:
        return np.zeros((0, SEQ_LEN, 51), np.float32), np.zeros((0,), np.int64), npz_path.name, npz_path.parent.name

    # Trim para video-level positivos (clips duros)
    if meta.get("video_level_label", 0) == 1 and TRIM_BORDERS > 0 and F > 2 * TRIM_BORDERS:
        y = y.copy()
        y[:TRIM_BORDERS] = 0
        y[-TRIM_BORDERS:] = 0

    # Features y visibilidad por frame
    feats = np.zeros((F, 51), dtype=np.float32)
    vis   = np.zeros(F, dtype=np.float32)
    for t in range(F):
        feats[t] = pool_frame_to_51(kps[t], W, H)
        vis[t]   = 1.0 if frame_visible(kps[t], conf_min=CONF_MIN) else 0.0

    # Ventanas con STRIDE, target = último frame
    Xw, Yw = [], []
    for s in range(0, F - SEQ_LEN + 1, STRIDE):
        e = s + SEQ_LEN
        if vis[s:e].mean() < MIN_VIS_FRAC:
            continue
        Xw.append(feats[s:e])
        Yw.append(int(y[e - 1]))  # tiempo real

    Xw = np.stack(Xw, 0) if Xw else np.zeros((0, SEQ_LEN, 51), np.float32)
    Yw = np.array(Yw, dtype=np.int64) if len(Yw) else np.zeros((0,), np.int64)
    return Xw, Yw, npz_path.name, npz_path.parent.name

def standardize_fit_apply(Xtr, Xva, Xte):
    Ntr, T, F = Xtr.shape
    mu = Xtr.reshape(-1, F).mean(axis=0, keepdims=True)
    sd = Xtr.reshape(-1, F).std(axis=0, keepdims=True) + 1e-6
    def norm(X):
        N = X.shape[0]
        return ((X.reshape(-1, F) - mu) / sd).reshape(N, T, F).astype("float32")
    return norm(Xtr), norm(Xva), norm(Xte), {"mean": mu.astype("float32"), "std": sd.astype("float32")}

def stratified_split_by_domain(vid_ids, domains, val_ratio, test_ratio, seed=SEED):
    """
    Split por VIDEO estratificado por dominio (carpeta padre).
    Retorna conjuntos de vid_ids: train_ids, val_ids, test_ids
    """
    rng = np.random.RandomState(seed)
    vid_ids = np.array(vid_ids)
    domains = np.array(domains)
    train_ids, val_ids, test_ids = set(), set(), set()
    for d in np.unique(domains):
        mask = (domains == d)
        vids_d = np.unique(vid_ids[mask])
        vids_d = vids_d.copy()
        rng.shuffle(vids_d)
        n = len(vids_d)
        n_test = int(round(n * test_ratio))
        n_val  = int(round(n * val_ratio))
        test_ids.update(vids_d[:n_test])
        val_ids.update(vids_d[n_test:n_test + n_val])
        train_ids.update(vids_d[n_test + n_val:])
    return train_ids, val_ids, test_ids

# =======================
# MODELOS
# =======================
def build_lstm(input_shape):
    reg = regularizers.l2(1e-4)
    inp = layers.Input(shape=input_shape)  # (T,51)
    x = layers.Masking(mask_value=0.0)(inp)
    x = layers.LSTM(64, return_sequences=True, dropout=0.3)(x)
    x = layers.SpatialDropout1D(0.1)(x)
    x = layers.LSTM(32, dropout=0.3)(x)
    x = layers.Dense(64, activation="relu", kernel_regularizer=reg)(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    return models.Model(inp, out)

def _conv_block(x, filters, reg):
    x = layers.Conv1D(filters, 3, padding="same", use_bias=False, kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LayerNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def build_cnn_lstm(input_shape):
    reg = regularizers.l2(1e-4)
    inp = layers.Input(shape=input_shape)  # (T,51)
    x = layers.Masking(mask_value=0.0)(inp)
    x = _conv_block(x, 64, reg)
    x = _conv_block(x, 64, reg)
    x = layers.Dropout(0.5)(x)
    x = layers.SpatialDropout1D(0.1)(x)
    x = layers.LSTM(64, return_sequences=True, dropout=0.3)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.LSTM(32, dropout=0.3)(x)
    x = layers.Dense(64, activation="relu", kernel_regularizer=reg)(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    return models.Model(inp, out)

# =======================
# MAIN
# =======================
if __name__ == "__main__":
    # 1) Cargar .npz y construir ventanas
    paths = []
    for d in NPZ_DIRS:
        paths += sorted(Path(d).glob("*.npz"))
    assert paths, "No se encontraron .npz en NPZ_DIRS"

    X_list, y_list, vid_ids, dom_list = [], [], [], []
    for p in paths:
        Xw, Yw, vid, dom = video_to_windows(p)
        if len(Xw):
            X_list.append(Xw); y_list.append(Yw)
            vid_ids += [vid] * len(Yw)
            dom_list += [dom] * len(Yw)

    if not X_list:
        raise RuntimeError("No se generaron ventanas (sube MIN_VIS_FRAC o revisa datos).")

    X_all = np.concatenate(X_list, 0).astype("float32")  # (N,T,51)
    y_all = np.concatenate(y_list, 0).astype("int32")    # (N,)
    print(f"Total ventanas: {len(y_all)} | Pos={y_all.sum()} ({y_all.mean():.3f})")

    # 2) Split por VIDEO estratificado por dominio
    vids_arr = np.array(vid_ids)
    doms_arr = np.array(dom_list)
    train_ids, val_ids, test_ids = stratified_split_by_domain(vids_arr, doms_arr, VAL_RATIO, TEST_RATIO, seed=SEED)

    tr_mask = np.isin(vids_arr, list(train_ids))
    va_mask = np.isin(vids_arr, list(val_ids))
    te_mask = np.isin(vids_arr, list(test_ids))

    X_tr, y_tr = X_all[tr_mask], y_all[tr_mask]
    X_va, y_va = X_all[va_mask], y_all[va_mask]
    X_te, y_te = X_all[te_mask], y_all[te_mask]

    print(f"Splits → train={len(y_tr)}  val={len(y_va)}  test={len(y_te)}")

    X_tr, X_va, X_te, stats = standardize_fit_apply(X_tr, X_va, X_te)

    # noise = np.random.normal(0, 0.01, size=X_tr.shape).astype("float32")
    # X_tr = np.clip(X_tr + noise, -3.0, 3.0)

    input_shape = (X_tr.shape[1], X_tr.shape[2])  # (SEQ_LEN, 51)
    model = build_lstm(input_shape) if MODEL_KIND == "lstm" else build_cnn_lstm(input_shape)
    model_name = f"mix_{MODEL_KIND}_T{SEQ_LEN}_F{input_shape[1]}"
    model.summary()

    try:
        AdamW = tf.keras.optimizers.AdamW
    except AttributeError:
        from tensorflow.keras.optimizers.experimental import AdamW
    opt = AdamW(learning_rate=3e-4, weight_decay=1e-4)

    model.compile(
        optimizer=opt,
        loss=BinaryFocalCrossentropy(gamma=1.5, label_smoothing=0.02),
        metrics=[
            tf.keras.metrics.BinaryAccuracy(name="acc"),
            tf.keras.metrics.AUC(name="auc"),
            tf.keras.metrics.AUC(name="pr_auc", curve="PR"),
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall"),
        ],
    )

    cbs = [
        callbacks.ModelCheckpoint(str(MODEL_DIR / f"{model_name}.keras"),
                                  save_best_only=True, monitor="val_pr_auc", mode="max"),
        callbacks.ReduceLROnPlateau(monitor="val_pr_auc", mode="max",
                                    factor=0.5, patience=4, verbose=1),
        callbacks.EarlyStopping(monitor="val_pr_auc", mode="max",
                                patience=8, restore_best_weights=True, verbose=1),
    ]

    hist = model.fit(
        X_tr, y_tr,
        validation_data=(X_va, y_va),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        class_weight=None,   # focal+AdamW
        callbacks=cbs,
        verbose=1,
    )

    y_val_prob = model.predict(X_va, batch_size=BATCH_SIZE, verbose=0).ravel()
    ts = np.linspace(0.05, 0.95, 19)
    best_f1, best_t = -1.0, 0.5
    for t in ts:
        y_pred = (y_val_prob >= t).astype(int)
        f1 = f1_score(y_va, y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, float(t)
    print(f"\nMejor umbral (val, F1): t={best_t:.2f}  F1={best_f1:.3f}")

    y_te_prob = model.predict(X_te, batch_size=BATCH_SIZE, verbose=0).ravel()
    y_te_pred = (y_te_prob >= best_t).astype(int)

    print("\n=== TEST ===")
    print("ROC-AUC :", roc_auc_score(y_te, y_te_prob))
    print("PR-AUC  :", average_precision_score(y_te, y_te_prob))
    print(classification_report(y_te, y_te_pred, digits=4))
    print(confusion_matrix(y_te, y_te_pred))

    np.savez(MODEL_DIR / f"{model_name}_norm_stats.npz", mean=stats["mean"], std=stats["std"])
    with open(MODEL_DIR / f"{model_name}_threshold.json", "w", encoding="utf-8") as f:
        json.dump({"best_threshold": best_t}, f, ensure_ascii=False, indent=2)

    print(f"\nGuardado modelo: {MODEL_DIR / f'{model_name}.keras'}")
    print(f"Guardado norm stats: {MODEL_DIR / f'{model_name}_norm_stats.npz'}")
    print(f"Guardado umbral óptimo: {MODEL_DIR / f'{model_name}_threshold.json'}")


Total ventanas: 9336 | Pos=2936 (0.314)
Splits → train=6715  val=1112  test=1509


c:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\venv\Lib\site-packages\keras\src\layers\layer.py:970: UserWarning: Layer 'conv1d_6' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 32, 51)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking_3 (Masking)             │ (None, 32, 51)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 32, 64)         │         9,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 32, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_6           │ (None, 32, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 32, 64)         │        12,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 32, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_7           │ (None, 32, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_3             │ (None, 32, 64)         │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 32, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,465 (275.25 KB)

 Trainable params: 70,209 (274.25 KB)

 Non-trainable params: 256 (1.00 KB)

Epoch 1/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - acc: 0.6721 - auc: 0.6123 - loss: 0.2349 - pr_auc: 0.3984 - precision: 0.4520 - recall: 0.1206 - val_acc: 0.7383 - val_auc: 0.5551 - val_loss: 0.2210 - val_pr_auc: 0.2747 - val_precision: 0.2857 - val_recall: 0.0599 - learning_rate: 3.0000e-04
Epoch 2/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - acc: 0.7029 - auc: 0.6917 - loss: 0.2231 - pr_auc: 0.4959 - precision: 0.5843 - recall: 0.2455 - val_acc: 0.7311 - val_auc: 0.5162 - val_loss: 0.2357 - val_pr_auc: 0.2451 - val_precision: 0.2576 - val_recall: 0.0637 - learning_rate: 3.0000e-04
Epoch 3/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - acc: 0.7239 - auc: 0.7295 - loss: 0.2149 - pr_auc: 0.5471 - precision: 0.6076 - recall: 0.3852 - val_acc: 0.7068 - val_auc: 0.4649 - val_loss: 0.2603 - val_pr_auc: 0.2272 - val_precision: 0.2243 - val_recall: 0.0899 - learning_rate: 3.0000e-04
Epoch 4/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - acc: 0.7375 - auc: 0.7568 - loss: 0.2080 - pr_auc: 0.588

In [105]:
import numpy as np
from pathlib import Path

# === CONFIG: agrega/quita carpetas con tus .npz ===
NPZ_DIRS   = [
    r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz",          # CHAD
    r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz_fall",     # FALL
    r"C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz_video"
]

TH = 0.5  # umbral para considerar positivo si las etiquetas vienen como float

def count_labels_in_file(npz_path: Path, th=TH):
    z = np.load(npz_path, allow_pickle=True)
    if "labels_aligned" not in z.files:
        return 0, 0, 0
    y = z["labels_aligned"]
    # soporta float/bool/int; convierte a 0/1 con umbral si hace falta
    if y.dtype.kind in "fc":
        y_bin = (y >= th).astype(np.int64)
    else:
        y_bin = y.astype(np.int64)
    pos = int(y_bin.sum())
    tot = int(y_bin.shape[0])
    neg = tot - pos
    return pos, neg, tot

def main():
    grand_pos = grand_neg = grand_tot = 0
    for d in NPZ_DIRS:
        d = Path(d)
        npzs = sorted(d.glob("*.npz"))
        if not npzs:
            print(f"[{d}] sin archivos .npz")
            continue

        dir_pos = dir_neg = dir_tot = 0
        print(f"\n=== Carpeta: {d} ===")
        for p in npzs:
            pos, neg, tot = count_labels_in_file(p)
            dir_pos += pos; dir_neg += neg; dir_tot += tot
            print(f"{p.name:<28} | frames={tot:6d}  pos={pos:6d}  neg={neg:6d}  pos_ratio={pos/ tot if tot else 0:.4f}")

        grand_pos += dir_pos; grand_neg += dir_neg; grand_tot += dir_tot
        print(f"-- Subtotal {d.name}: frames={dir_tot}  pos={dir_pos}  neg={dir_neg}  pos_ratio={(dir_pos/dir_tot if dir_tot else 0):.4f}")

    print("\n=== TOTAL GLOBAL ===")
    print(f"frames={grand_tot}  pos={grand_pos}  neg={grand_neg}  pos_ratio={(grand_pos/grand_tot if grand_tot else 0):.4f}")

if __name__ == "__main__":
    main()



=== Carpeta: C:\Users\Usuario\Documents\GitHub\tesis-deteccionar-sistema\out_npz ===
1_003_0.npz                  | frames=  6299  pos=     0  neg=  6299  pos_ratio=0.0000
1_033_0.npz                  | frames=  1799  pos=     0  neg=  1799  pos_ratio=0.0000
1_069_1.npz                  | frames=  1199  pos=   877  neg=   322  pos_ratio=0.7314
1_070_1.npz                  | frames=   449  pos=   198  neg=   251  pos_ratio=0.4410
1_071_1.npz                  | frames=   599  pos=   322  neg=   277  pos_ratio=0.5376
1_073_1.npz                  | frames=   449  pos=   150  neg=   299  pos_ratio=0.3341
1_076_1.npz                  | frames=   449  pos=   121  neg=   328  pos_ratio=0.2695
1_078_1.npz                  | frames=  1199  pos=   651  neg=   548  pos_ratio=0.5430
1_082_1.npz                  | frames=   449  pos=   159  neg=   290  pos_ratio=0.3541
1_083_1.npz                  | frames=   449  pos=   212  neg=   237  pos_ratio=0.4722
1_084_1.npz                  | frames=  1199